In [ ]:
!pip install --upgrade pyTelegramBotAPI
# Setup the environment
!pip install -q -U immutabledict sentencepiece
!pip install -U transformers datasets
!pip install ipywidgets
!pip install telebot
!pip install python-telegram-bot

In [ ]:
import os
import telebot
import json
# Create a folder to save chat history
os.makedirs('chats', exist_ok=True)

bot = telebot.TeleBot("your telegram api")


import traceback

In [ ]:
def translate(chat):
    try:
        query = f'''ترجم الي اللهجه المصريه: "{chat}"'''
        prompt_template = """<start_of_turn>user يوجد أدناه تعليمات تصف المهمة. اكتب الرد الذي يكمل الطلب بشكل مناسب.\n\n{query}<end_of_turn>\n<start_of_turn>model"""
        prompt = prompt_template.format(query=query)

        print(f"Tokenizing prompt: {prompt}")
        inputs = tokenizer(prompt, return_tensors="pt", add_special_tokens=True)
        print("Tokenization successful.")
        print(f"Input IDs shape: {inputs['input_ids'].shape}")
        print(f"Attention mask shape: {inputs['attention_mask'].shape}")

        model_inputs = inputs.to('cuda')

        print("Generating translation...")
        outputs = model.generate(input_ids=model_inputs['input_ids'], attention_mask=model_inputs['attention_mask'], max_new_tokens=1000)
        print("Generation successful.")
        print(f"Output shape: {outputs.shape}")

        translation = tokenizer.decode(outputs[0], skip_special_tokens=True)
        print(f"Decoded translation: {translation}")

        return translation.split('model')[-1].strip()
    except Exception as e:
        print(f"Error in translate function: {str(e)}")
        print(traceback.format_exc())
        return f"An error occurred: {str(e)}"

In [ ]:
def get_chats(user_id):
    path = f'chats/{user_id}.json'
    if os.path.exists(path):
        with open(path, 'r') as file:
            data = json.load(file)
        return data['chat']
    else:
        return []

In [ ]:
def write_chat(new_data, user_id):
    path = f'chats/{user_id}.json'
    if os.path.exists(path):
        with open(path, 'r+') as file:
            file_data = json.load(file)
            file_data["chat"].append(new_data)
            file.seek(0)
            json.dump(file_data, file, indent=4)
    else:
        data = {"user_id": user_id, "chat": [new_data]}
        with open(path, 'w') as file:
            json.dump(data, file, indent=4)

In [ ]:
def handle_chats(user_id, message):
    write_chat({"role": "user", "content": message}, user_id)
    print(f"User: {message}")

    translation = translate(message)
    print(f"Translator: {translation}")

    write_chat({"role": "assistant", "content": translation}, user_id)
    return translation

In [ ]:
@bot.message_handler(commands=['start'])
def send_welcome(message):
    bot.reply_to(message, "مرحبا! أنا مترجم اللهجة المصرية. أرسل لي أي نص بالإنجليزية وسأترجمه إلى اللهجة المصرية.")

@bot.message_handler(func=lambda message: True)
def translate_message(message):
    try:
        user_id = str(message.from_user.id)
        print(f"Received message from user {user_id}: {message.text}")
        translation = handle_chats(user_id, message.text)
        bot.reply_to(message, translation)
    except Exception as e:
        error_message = f"An error occurred: {str(e)}"
        print(error_message)
        print(traceback.format_exc())
        bot.reply_to(message, error_message)

In [ ]:
if __name__ == '__main__':
    print("Bot is running...")
    bot.polling()